지난 글 [스트림 인터페이스](http://wikibootup.github.io/sicp/3-2-stream-interface.html)에서는 연산을 필요한 만큼만 계산하는 스트림의 개념을 살펴보았고,

스트림을 만들기 위한 기본 인터페이스를 만들어보았습니다. 

여기서는 만들었던 인터페이스를 이용해

1. __무한스트림__을 만들어보고,
2. 무한스트림 방식으로 팩토리얼 구하기 및 스트림의 부분합 구하기를 해보겠습니다.
3. 그리고, 근사값 구하기 문제를 스트림 가속기를 이용하여 풀어보겠습니다.

저번 글에서 `pair` 타입으로 `stream`을 만들었습니다.

쌍의 첫번째에는 현재 값이, 두번째에는 다음 값에 대한 '약속'이 들어있었습니다.

만약, 어떤 스트림에 __끝 없이 다음 값이 존재한다면__ 그 스트림을 무한스트림이라고 합니다.

'게으르지 않는' 프로그램에서는 한 번 명령하면 모든 연산을 한 방에 끝내려고 하기 떄문에,

'무한 재귀 함수' 같은 경우 `Maximum recursion error`와 같은 오류를 일으킵니다.

하지만 스트림 연산은 '한 번에 한 계산' 씩 수행하므로, 이런 오류를 피할 수 있습니다.

In [1]:
from modules.basic import cons_stream, stream_map

In [19]:
def integers_starting_from(n):
    cons_stream(n, integers_starting_from(n))

In [20]:
integers_starting_from(1)

RuntimeError: maximum recursion depth exceeded

In [11]:
cons_stream(1, 2)

(1, <generator object promise at 0x104d23510>)